### 安裝套件

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

### 載入資料

In [9]:
data = pd.read_csv("/Users/wangjigang/Desktop/數據競賽/2024-pre-train/6/c.csv")
data_test = pd.read_csv("/Users/wangjigang/Desktop/數據競賽/2024-testing.csv")

### 模型預測函式

In [51]:
def predict(col):
    ### 所需資料 ###
    dt = pd.concat([data["y01"], data["y02"], data["y03"], data["y04"], data["y05"], 
                    data["y06"], data["y07"], data["y08"], data["y09"], data["y10"],
                    data_test[col][:50]]).reset_index(drop=True)
    test_c = data_test[col][:50]

    ### 模型參數 ###
    parameter = [(2, 0, 0, 500), (2, 0, 0, 1000), (2, 0, 4, 500), (2, 0, 4, 500), (2, 0, 4, 950)]

    ### 第一次 ###
    model = ARIMA(dt, order=(1, 0, 0)) 
    forecast = model.fit().forecast(steps=3950)
    test_c = pd.concat([test_c, forecast[:500]]).reset_index(drop=True)
    dt = pd.concat([data["y01"][len(test_c):], data["y02"][len(test_c):], data["y03"][len(test_c):], data["y04"][len(test_c):], data["y05"][len(test_c):], 
                    data["y06"][len(test_c):], data["y07"][len(test_c):], data["y08"][len(test_c):], data["y09"][len(test_c):], data["y10"][len(test_c):],
                    test_c[len(test_c)-50:]]).reset_index(drop=True)
    ### 後五次 ###
    for pdq in parameter:
        p = pdq[0]
        d = pdq[1]
        q = pdq[2]
        time = pdq[3]
        model = ARIMA(dt, order=(p, d, q)) 
        forecast = model.fit().forecast(steps=len(test_c))
        test_c = pd.concat([test_c, forecast[:time]]).reset_index(drop=True)
        dt = pd.concat([data["y01"][len(test_c):], data["y02"][len(test_c):], data["y03"][len(test_c):], data["y04"][len(test_c):], data["y05"][len(test_c):], 
                       data["y06"][len(test_c):], data["y07"][len(test_c):], data["y08"][len(test_c):], data["y09"][len(test_c):], data["y10"][len(test_c):],
                       test_c[len(test_c)-50:]]).reset_index(drop=True)
    # plt.plot(test_c)
    # test_c.to_excel(f"/Users/wangjigang/Desktop/數據競賽/預測結果/predict_{col}.xlsx", index = False)
    return test_c

### 進行預測

In [52]:
p_y01 = predict("y01")
p_y02 = predict("y02")
p_y03 = predict("y03")
p_y04 = predict("y04")
p_y05 = predict("y05")
p_y06 = predict("y06")
p_y07 = predict("y07")
p_y08 = predict("y08")
p_y09 = predict("y09")
p_y10 = predict("y10")

### 合併預測結果

In [53]:
results = pd.concat([p_y01, p_y02, p_y03, p_y04, p_y05, 
                     p_y06, p_y07, p_y08, p_y09, p_y10,
                     ], axis = 1).reset_index(drop=True)

### 輸出資料

In [61]:
results.to_excel("./predict_results.xlsx", index = False)